In [1]:
!pip install requests


In [44]:
from getpass import getpass
import requests

# Prompt user for key (hidden input)
API_KEY = getpass("Enter your API key: ")

MODEL = "models/gemini-2.0-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

def ask_llm(prompt, temperature=0.7, max_output_tokens=200):
    headers = {
        "Content-Type": "application/json",
        "X-goog-api-key": API_KEY
    }

    data = {
        "contents": [
            {"role": "user", "parts": [{"text": prompt}]}
        ],
        "generationConfig": {"temperature": temperature, "maxOutputTokens": max_output_tokens}
    }

    response = requests.post(URL, headers=headers, json=data)
    if response.status_code != 200:
        return f"Error {response.status_code}: {response.text}"

    resp_json = response.json()
    try:
        candidates = resp_json.get("candidates", [])
        if not candidates:
            return "No candidates returned."

        parts = candidates[0].get("content", {}).get("parts", [])
        if not parts:
            return "No text generated."

        return parts[0].get("text", "").strip()
    except Exception as e:
        return f"Error parsing response: {e}\nRaw: {resp_json}"


Enter your API key: ··········


In [45]:
prompt = "Explain how AI works in simple words."
print(ask_llm(prompt))


Imagine you're teaching a dog a trick. You show them what you want, and if they do it right, you give them a treat. They learn by seeing examples and getting feedback.

AI is similar, but instead of a dog, it's a computer program, and instead of treats, it gets "data."

Here's the breakdown:

*   **Data:** AI learns from massive amounts of information (like text, images, or numbers). Think of this as the examples you show the dog.
*   **Algorithms:** These are the "instructions" or "rules" the AI follows to analyze the data and find patterns.
*   **Learning:** The AI uses these algorithms to look for patterns in the data.  It tries to predict or make decisions based on what it's learned.  If it gets it right (like the dog doing the trick), it strengthens those patterns. If it gets it wrong, it adjusts the patterns until it


In [51]:
##Zero-shot Prompting
prompt = "Summarize the following text in one sentence:\nAI is changing the way people work and learn."
print(ask_llm(prompt))


AI is revolutionizing both the professional and educational landscapes.


In [52]:
##Few-shot Prompting
prompt = """Summarize the following examples:
Example 1: "AI helps doctors analyze data." → "AI assists medical professionals."
Example 2: "Self-driving cars use AI to navigate." → "AI enables autonomous vehicles."
Now summarize: "AI is transforming education by personalizing learning."""
print(ask_llm(prompt))


AI personalizes learning in education.


In [53]:
##role Prompting
prompt = "Act as a data scientist. Explain how AI predicts stock prices in simple terms."
print(ask_llm(prompt))


Alright, let's break down how AI tries to predict stock prices. Think of it like this: instead of just looking at the news or some basic charts, AI uses *everything* it can get its hands on to try and see patterns that humans might miss.

Here's the simplified version:

**1. Gathering the Data - The More, The Merrier!**

*   **Historical Stock Data:** This is the bread and butter.  AI looks at past stock prices, trading volume (how many shares were bought and sold), and other technical indicators like moving averages (which smooth out price fluctuations) over a long period.  Think of it like studying the past behavior of a stock to see if history tends to repeat itself.
*   **Financial News & Sentiment:** AI reads news articles, financial reports, social media posts, and even analyzes the tone (sentiment) of the language used.  Is the news positive or negative about a company or the overall economy


In [54]:
##structured Response
prompt = """List three benefits of AI in bullet points:
-"""
print(ask_llm(prompt))


Here are three benefits of AI in bullet points:

*   **Increased Efficiency and Productivity:** AI can automate repetitive tasks, analyze large datasets quickly, and optimize processes, leading to significant improvements in efficiency and productivity across various industries.
*   **Improved Decision-Making:** AI algorithms can identify patterns and insights from data that humans might miss, leading to more informed and data-driven decision-making in areas like business strategy, healthcare, and finance.
*   **Enhanced Personalization and User Experience:** AI can personalize products, services, and content based on individual user preferences and behaviors, leading to more engaging and satisfying experiences.


In [55]:
def test_prompt(prompt, temperature=0.7):
    print("PROMPT:")
    print(prompt)
    print("\nRESPONSE:")
    print(ask_llm(prompt, temperature=temperature))
    print("\n" + "-"*50 + "\n")


In [56]:
test_prompt("Summarize this text in one sentence:\nAI is transforming work and learning.")
test_prompt("Act as a teacher. Explain quantum computing in simple terms.")
test_prompt("""Solve step by step: 45 * 12""")


PROMPT:
Summarize this text in one sentence:
AI is transforming work and learning.

RESPONSE:
Artificial intelligence is fundamentally changing both the nature of work and the methods of education.

--------------------------------------------------

PROMPT:
Act as a teacher. Explain quantum computing in simple terms.

RESPONSE:
Alright class, settle down, settle down! Today we're going to talk about something super cool, something that might sound like science fiction but is very real: **quantum computing!**

Now, before you start picturing robots and teleportation, let's break down what it actually *is*.

Think of it this way: **regular computers, the ones you use every day, are like light switches.** They can be either **ON (representing a 1) or OFF (representing a 0).** That's how they store and process information. This basic unit of information is called a **bit.**

Quantum computers, on the other hand, are like **dimmer switches.** They're not just ON or OFF, they can be **ON, O

**Prompt Library**

In [57]:
PROMPT_LIBRARY = {
    "summarization": "Summarize the following text in one concise sentence:\n{text}",

    "code_explanation": "Explain the following Python code in simple words for a beginner:\n{code}",

    "debug_code": "Find and explain any bugs in this Python code, and provide a corrected version:\n{code}",

    "brainstorm_ideas": "Give me 5 creative ideas for {topic}. Be original and specific.",

    "role_prompt_data_scientist": "Act as a data scientist and explain how {topic} works in simple terms.",

    "step_by_step_reasoning": "Solve the following problem step by step, showing your reasoning:\n{problem}",

    "structured_bullets": "List 5 key points about {topic} in bullet format.",

    "tone_adjustment": "Rewrite the following text in a {tone} tone:\n{text}",

    "compare_versions": "Compare the two pieces of text and summarize the differences:\nText 1: {text1}\nText 2: {text2}",

    "one_shot_example": """Example task: Translate English to French.
English: "Hello, how are you?" → French: "Bonjour, comment ça va?"
Now translate: "{text}" """,

    "few_shot_example": """Here are examples of summarization:
Example 1: "AI helps doctors analyze data." → "AI assists medical professionals."
Example 2: "Self-driving cars use AI to navigate." → "AI enables autonomous vehicles."
Now summarize: "{text}" """
}


In [58]:
def use_prompt(prompt_name, **kwargs):
    """
    Retrieves a prompt template from the library, fills in variables, and sends to LLM.
    """
    if prompt_name not in PROMPT_LIBRARY:
        return f"Prompt '{prompt_name}' not found in library."

    prompt_template = PROMPT_LIBRARY[prompt_name]
    prompt = prompt_template.format(**kwargs)

    return ask_llm(prompt)


In [59]:
# 1. Summarization
text = "Artificial intelligence is transforming the way people work, learn, and communicate."
print(use_prompt("summarization", text=text))

# 2. Code explanation
code = "def add(a, b):\n    return a + b"
print(use_prompt("code_explanation", code=code))

# 3. Brainstorm ideas
print(use_prompt("brainstorm_ideas", topic="AI startup ideas"))

# 4. Step-by-step reasoning
problem = "If a car travels 80 km/h for 3 hours, how far does it go?"
print(use_prompt("step_by_step_reasoning", problem=problem))

# 5. Role prompt
print(use_prompt("role_prompt_data_scientist", topic="predicting stock prices"))

# 6. Structured bullet points
print(use_prompt("structured_bullets", topic="benefits of AI in healthcare"))


AI is revolutionizing work, education, and communication.
Okay, let's break down this Python code:

**What it does:**

This code defines a simple function that adds two numbers together.  That's it!

**Explanation, piece by piece:**

1. **`def add(a, b):`**
   * **`def`**: This keyword tells Python that you are *defining* a function.  Think of it like creating a named set of instructions.
   * **`add`**: This is the *name* of the function.  You can choose almost any name you want (it should be descriptive). In this case, "add" is a good name because it suggests what the function does: addition.
   * **`(a, b)`**: These are the *parameters* of the function.  Parameters are like placeholders or input values that the function needs to do its job.
      * `a` and `b` are just variable names.  They will
Okay, here are 5 creative and specific AI startup ideas, aiming for originality and focusing on niche areas:

1.  **AI-Powered Personalized Dream Journaling & Interpretation (Project: Oneiro